In [6]:
from classes import Neuron, Layer, Network_Model
import helpers as hlp
import pandas as pd
import math
import random as rnd

In [104]:
df_orig = pd.read_excel('datasets\\rgb.xlsx')
df = df_orig.copy()
df

,Name,R,G,B,Beige,Yellow,Orange,Red,Purple,Blue,Green,Grey,Brown,White,Black
0,Beige,138,102,66,1,0,0,0,0,0,0,0,0,0,0
1,Beige,169,131,7,1,0,0,0,0,0,0,0,0,0,0
2,Beige,190,189,127,1,0,0,0,0,0,0,0,0,0,0
3,Beige,194,176,120,1,0,0,0,0,0,0,0,0,0,0
4,Beige,198,166,100,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Yellow,244,169,0,0,1,0,0,0,0,0,0,0,0,0
209,Yellow,245,208,51,0,1,0,0,0,0,0,0,0,0,0
210,Yellow,248,243,53,0,1,0,0,0,0,0,0,0,0,0
211,Yellow,250,210,1,0,1,0,0,0,0,0,0,0,0,0


In [105]:
df_clean = df.drop(columns=['Name'])
data = df_clean.values.tolist()
data

[[138, 102, 66, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [169, 131, 7, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [190, 189, 127, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [194, 176, 120, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [198, 166, 100, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [199, 180, 70, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [205, 164, 52, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [220, 156, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [225, 204, 79, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [228, 160, 16, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [229, 190, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [230, 214, 144, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [234, 230, 202, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [10, 10, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [28, 28, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [30, 30, 30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [40, 40, 40, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [2, 86, 105, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [6, 57, 113, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [14, 41, 75, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [16, 44, 84